In [60]:
import pprint
from torchtext.vocab import FastText,vocab
import json,csv
import pandas as pd
from nltk.tokenize import RegexpTokenizer,TweetTokenizer
import re
from collections import defaultdict
from tqdm import tqdm

import torch
from tqdm import tqdm

In [71]:
import numpy as np

In [65]:
import pickle

In [2]:
!wget https://raw.githubusercontent.com/Ellana42/style_transfer_proust/main/datasets/dataset.csv

--2022-04-23 10:09:17--  https://raw.githubusercontent.com/Ellana42/style_transfer_proust/main/datasets/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47153844 (45M) [text/plain]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>]  44.97M   201MB/s    in 0.2s    

2022-04-23 10:09:18 (201 MB/s) - ‘dataset.csv’ saved [47153844/47153844]



#Create embedding with less vocabulary from fasttext embedding


In [ ]:
with open("/content/dataset.csv","r",encoding='utf-8') as f:
  data=csv.reader(f)
  texts=[]
  for row in data:
    texts.append(row)

In [5]:
df = pd.read_csv('/content/dataset.csv', sep='|', index_col=0).dropna()

In [6]:
df.head()

,text,label
0,"Les ""grands dangers"" qui guettent les voyageur...",0
1,\nOn prête au premier ministre de l'État d'I...,0
2,L e cortège prend des allures de visite guidée...,0
3,U ne pluie de graines et d’informations s’est ...,0
4,Fiscalité du patrimoine : ce que proposent les...,0


In [7]:
tokenizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

def tokenize_sentence(sentence: str,tokenizer: RegexpTokenizer):
  '''Simple tokenizer, removes or replaces special characters
  sentence : str sentence to be tokenized
  tokenizer : tokenizer with tokenize method '''

  #Lower capital leters
  tokenized=sentence.lower()
  #Change special character
  tokenized=re.sub("’","'",tokenized)
  #Remove unwanted characters
  tokenized=re.sub("(@\w*\\b\s?|#\w*\\b\s?|&\w*\\b\s?|\n\s?|\\\\|\<|\>|\||\*)","",tokenized)
  tokenized=re.sub("\/","",tokenized)
  #Replace articles since model does not embed contractions well
  tokenized=re.sub("l'","le ",tokenized)
  tokenized=re.sub("d'","de ",tokenized)
  tokenized=re.sub("j'","je ",tokenized)
  tokenized=re.sub("qu'","que ",tokenized)
  tokenized=re.sub("t'","te ",tokenized)
  tokenized=re.sub("c'","ce ",tokenized)
  #Tokenize sentence
  tokenized=tokenizer.tokenize(tokenized)
  return(tokenized)

In [8]:
frequency_dict=defaultdict(int)
for i in tqdm(range(len(df))):
  text=df.iloc[i]["text"]
  tokens=tokenize_sentence(text,tokenizer)
  for token in tokens:
    frequency_dict[token]+=1

100%|██████████| 20263/20263 [00:11<00:00, 1753.91it/s]


In [13]:
list_dict=list(frequency_dict.items())
sorted_dict=dict(sorted(list_dict, key = lambda a : a[1],reverse=True))

In [15]:
len(sorted_dict)

135435

In [23]:
list(sorted_dict.keys())[35000]

'automnal'

In [24]:
sorted_dict["automnal"]

8

In [14]:
print(sorted_dict)

{'de': 497327, ',': 480919, '.': 346663, 'le': 331033, 'la': 205857, 'à': 159895, '-': 134400, 'et': 130203, 'les': 125833, 'que': 110308, 'en': 106647, '"': 104854, 'des': 102399, 'un': 98726, 'a': 97464, 'une': 79251, 'du': 76769, 'il': 73032, 'est': 69881, 'pour': 67761, 'dans': 62278, 'ce': 62061, 'qui': 60157, 'sur': 49196, 'au': 47475, 'pas': 47063, ':': 46955, 'par': 45245, '(': 44548, ')': 44539, 'je': 41091, 'plus': 37132, 'ne': 34237, '»': 32779, "n'": 31760, 'son': 31183, 'avec': 30539, "s'": 29774, 'se': 28447, 'elle': 27950, 'mais': 26352, '«': 25133, 'on': 24062, 'y': 22897, 'ont': 22049, 'été': 21399, 'avait': 20762, 'sa': 19926, 'était': 18800, 'nous': 18693, 'cette': 18374, 'comme': 17946, 'aux': 17879, 'sont': 17421, 'vous': 17284, 'ses': 16318, 'ou': 16301, 'être': 15648, 'même': 15182, 'si': 14744, 'après': 14617, 'lui': 14484, 'tout': 14391, '%': 14128, 'fait': 14113, '?': 13805, 'deux': 13532, 'aussi': 12962, 'leur': 12382, 'ces': 11991, '2016': 11545, 'ans': 1141

In [58]:
filtered_vocab = dict((key, freq) for key, freq in sorted_dict.items() if freq >= 10)

#restrict fasttext vocab

In [32]:
ft_vectors = FastText(language='fr')
#vocab takes a counter as argument, it keeps the order of the dictionnary and sets the indexes according to these indexes.
#vocab ignores all the words with 0 occurences, since here what we have is an stoi dictionnary and not a counter
#We have to give value at least 1 to the element which was assigned index 0 other wise it won't be included in the vocab object
ft_vectors.stoi=dict((key, value+1) for key, value in ft_vectors.stoi.items())
ft_vectors.stoi
ft_vocab=vocab(ft_vectors.stoi)

.vector_cache/wiki.fr.vec: 3.03GB [01:12, 41.5MB/s]                            
100%|██████████| 1152449/1152449 [02:11<00:00, 8731.21it/s]


In [36]:
list(ft_vectors.stoi.items())[-1]

('concentrat', 1152449)

In [62]:
def get_new_embedding(word_dict, min_freq, ft_vectors, ft_vocab,filter):
  if filter:
    filtered_word_dict = dict((key, freq) for key, freq in word_dict.items() if freq >= min_freq)
  else:
    filtered_word_dict = word_dict
  matching_vocabs = list(set(filtered_word_dict).intersection(set(ft_vectors.stoi)))

  new_stoi={"</s>" : 1 }
  new_embedding_vect = torch.tensor(ft_vectors["</s>"]).reshape(1,-1)

  for k in tqdm(range(len(matching_vocabs))):
    curr_word = matching_vocabs[k]
    new_stoi[matching_vocabs[k]]=k+2
    embed_vect = torch.tensor(ft_vectors[curr_word]).reshape(1,-1)
    new_embedding_vect = torch.cat((new_embedding_vect, embed_vect))
  return(new_stoi,new_embedding_vect)

In [64]:
new_stoi, new_embedding = get_new_embedding(filtered_vocab, 10, ft_vectors, ft_vocab, False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
  0%|          | 0/30755 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 30755/30755 [03:03<00:00, 167.35it/s]


In [97]:
new_itos=list(new_stoi.keys())

In [74]:
torch.save(new_embedding,"/content/embedding.pt")

In [75]:
embed = torch.load("/content/embedding.pt")

In [82]:
with open("/content/soit.pickle","wb") as f:
  pickle.dump(new_stoi,f)
with open("/content/itos.pickle","wb") as f:
  pickle.dump(new_itos,f)

In [84]:
with open("/content/soit.pickle","rb") as f:
  a=pickle.load(f)
with open("/content/itos.pickle","rb") as f:
  b=pickle.load(f)